# Data Formatting Script

Importing the necessary packages

In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
conf = pd.read_csv("data/raw_data/ucdp-prio-acd-181.csv")  # reading in conflict dyads data
conf.head()  # examining its structure

,conflict_id,location,side_a,side_a_id,side_a_2nd,side_b,side_b_id,side_b_2nd,incompatibility,territory_name,...,ep_end,ep_end_date,ep_end_prec,gwno_a,gwno_a_2nd,gwno_b,gwno_b.1,gwno_loc,region,version
0,200,Bolivia,Government of Bolivia,23,NaN,Popular Revolutionary Movement,719,NaN,2,NaN,...,1,1946-07-21,1.0,145,NaN,NaN,NaN,145,5,18.1
1,200,Bolivia,Government of Bolivia,23,NaN,MNR,720,NaN,2,NaN,...,1,1952-04-12,1.0,145,NaN,NaN,NaN,145,5,18.1
2,200,Bolivia,Government of Bolivia,23,NaN,ELN,721,NaN,2,NaN,...,1,1967-10-16,1.0,145,NaN,NaN,NaN,145,5,18.1
3,201,Cambodia (Kampuchea),Government of France,33,NaN,Khmer Issarak,160,NaN,1,Cambodia,...,0,NaN,NaN,220,NaN,NaN,NaN,811,3,18.1
4,201,Cambodia (Kampuchea),Government of France,33,NaN,Khmer Issarak,160,NaN,1,Cambodia,...,0,NaN,NaN,220,NaN,NaN,NaN,811,3,18.1


In [3]:
# Function to convert numeric values and return 0 if they are not numeric, 
# this will allow entries without countries as the co-belligerent to be filtered out
def integerize(x):
    try:
        return int(x)
    except ValueError:
        return 0
    
# Filter the data to just that have countries on both sides
country_conf = conf[conf.gwno_b.apply(integerize) > 0]
print(country_conf.shape) # shape of the data frame
print(country_conf.columns) # names of the columns
country_conf.head()

(125, 28)
Index(['conflict_id', 'location', 'side_a', 'side_a_id', 'side_a_2nd',
       'side_b', 'side_b_id', 'side_b_2nd', 'incompatibility',
       'territory_name', 'year', 'intensity_level', 'cumulative_intensity',
       'type_of_conflict', 'start_date', 'start_prec', 'start_date2',
       'start_prec2', 'ep_end', 'ep_end_date', 'ep_end_prec', 'gwno_a',
       'gwno_a_2nd', 'gwno_b', 'gwno_b.1', 'gwno_loc', 'region', 'version'],
      dtype='object')


,conflict_id,location,side_a,side_a_id,side_a_2nd,side_b,side_b_id,side_b_2nd,incompatibility,territory_name,...,ep_end,ep_end_date,ep_end_prec,gwno_a,gwno_a_2nd,gwno_b,gwno_b.1,gwno_loc,region,version
119,214,"France, Thailand",Government of France,33,NaN,Government of Thailand,147,NaN,1,Northern Cambodia,...,1,1946-11-17,1.0,220,NaN,800,NaN,"220, 800","1, 3",18.1
120,215,"Albania, United Kingdom",Government of Albania,45,NaN,Government of United Kingdom,28,NaN,1,Korfu Channel,...,1,1946-12-31,5.0,339,NaN,200,NaN,"200, 339",1,18.1
131,218,"India, Pakistan",Government of India,141,NaN,Government of Pakistan,142,NaN,1,Kashmir,...,1,1948-12-31,1.0,750,NaN,770,NaN,"750, 770",3,18.1
132,218,"India, Pakistan",Government of India,141,NaN,Government of Pakistan,142,NaN,1,Kashmir,...,0,NaN,NaN,750,NaN,770,NaN,"750, 770",3,18.1
133,218,"India, Pakistan",Government of India,141,NaN,Government of Pakistan,142,NaN,1,Kashmir,...,1,1965-12-15,1.0,750,NaN,770,NaN,"750, 770",3,18.1


In [4]:
print(country_conf['gwno_a']) # examine the structure when there are multiple gwno's

119                         220
120                         339
131                         750
132                         750
133                         750
134                         750
135                         750
136                         750
137                         750
138                         750
139                         750
140                         750
141                         750
142                         750
143                         750
144                         750
145                         750
146                         750
147                         750
148                         750
149                         750
150                         750
151                         750
152                         750
314                         751
348     651, 645, 663, 660, 652
349     651, 645, 663, 660, 652
424                         710
425                         710
426                         710
                 ...           
1567    

In [5]:
nodes = set()  # set to store the unique countries in the data
country_name_gwno_mapping = {} # dictionary to store the combinations of gwno's and country names


for i, row in country_conf.iterrows(): # for every row in the data
    for num, name in zip(row['gwno_a'].split(','), row['side_a'].split(',')): # grab the gwno and country names
        if num not in country_name_gwno_mapping:
            country_name_gwno_mapping[num]= name.strip()
        nodes.add(num.strip())
        
    for num, name in zip(row['gwno_b'].split(','), row['side_b'].split(',')): # grab the gwno and country names
        if num not in country_name_gwno_mapping:
            country_name_gwno_mapping[num]= name.strip()
        nodes.add(num.strip())

In [6]:
print(nodes)
# print(country_name_gwno_mapping)
for k, v in country_name_gwno_mapping.items():
    print(k, v)

{'850', '432', '522', '680', '751', '439', '820', '160', '500', '615', '471', '93', '816', '2', '770', '95', '750', '200', '520', '800', '811', '663', '310', '626', '660', '900', '130', '713', '365', '640', '690', '812', '135', '91', '475', '731', '92', '55', '645', '620', '531', '483', '510', '220', '339', '616', '600', '817', '210', '530', '775', '651', '732', '666', '678', '710', '630', '625', '700', '652', '352'}
220 Government of France
800 Government of Thailand
339 Government of Albania
200 Government of United Kingdom
750 Government of India
770 Government of Pakistan
751 Government of Hyderabad
651 Government of Egypt
 645 Government of Iraq
 663 Government of Jordan
 660 Government of Lebanon
 652 Government of Syria
666 Government of Israel
710 Government of China
713 Government of Taiwan
731 Government of North Korea
732 Government of South Korea
310 Government of Hungary
365 Government of Russia (Soviet Union)
91 Government of Honduras
93 Government of Nicaragua
530 Govern

In [7]:
conflicts = {} # dictionary to hold the combinations of countries
alliances = {}

for i, row in country_conf.iterrows():
    # List of all of the countries on side a
    side_a = [num.strip() for num in row['gwno_a'].split(',')]
    
    # List of all the countries on side b
    side_b = [num.strip() for num in row['gwno_b'].split(',')]
    
    for i in side_a:
        if i not in alliances:
            alliances[i] = {}
        for j in side_a:
            if i != j:
                if j in alliances[i]:
                    alliances[i][j] += 1
                else:
                    alliances[i][j] = 1
                    
    for i in side_b:
        if i not in alliances:
            alliances[i] = {}
        for j in side_a:
            if i != j:
                if j in alliances[i]:
                    alliances[i][j] += 1
                else:
                    alliances[i][j] = 1
    
    for num_a in side_a:
        if num_a not in conflicts:
            conflicts[num_a] = {}
        for num_b in side_b:
            if num_b in conflicts[num_a]:
                conflicts[num_a][num_b] += 1
#                 links[num_a][num_b][1].append(row['start_date'])
            else:
                conflicts[num_a][num_b] = 1
#                 links[num_a][num_b] = [1, [row['start_date']]]
                
    for num_b in side_b:
        if num_b not in conflicts:
            conflicts[num_b] = {}
        for num_a in side_a:
            if num_a in conflicts[num_b]:
                conflicts[num_b][num_a] += 1
            else:
                conflicts[num_b][num_a] = 1
    

In [8]:
print(conflicts)
print(alliances)

{'220': {'800': 1, '616': 1}, '800': {'220': 1, '811': 3, '812': 3}, '339': {'200': 1}, '200': {'339': 1, '651': 2, '160': 1, '645': 1}, '750': {'770': 22, '751': 1, '710': 2}, '770': {'750': 22}, '751': {'750': 1}, '651': {'666': 6, '200': 2}, '645': {'666': 2, '630': 10, '690': 2, '900': 1, '200': 1, '2': 1}, '663': {'666': 3}, '660': {'666': 2}, '652': {'666': 4}, '666': {'651': 6, '645': 2, '663': 3, '660': 2, '652': 4}, '710': {'713': 4, '750': 2, '775': 1, '365': 1, '816': 10}, '713': {'710': 4}, '731': {'732': 5}, '732': {'731': 5}, '310': {'365': 1}, '365': {'310': 1, '710': 1, '700': 1}, '91': {'93': 1, '92': 1}, '93': {'91': 1}, '530': {'520': 4, '531': 4}, '520': {'530': 4}, '616': {'220': 1}, '850': {'210': 1, '820': 4}, '210': {'850': 1}, '615': {'600': 1}, '600': {'615': 1}, '820': {'850': 4}, '817': {'816': 11}, '816': {'817': 11, '811': 3, '710': 10}, '811': {'800': 3, '816': 3}, '775': {'710': 1}, '92': {'91': 1}, '680': {'678': 2}, '678': {'680': 2}, '352': {'640': 1}

In [9]:
nodes_json = [{'id': i}for i in nodes]
print(nodes_json)
conflicts_json = [{'source': a, 'target': b, 'value': w , 'type': 'enemy'} for a, v in conflicts.items() for b, w in v.items() ]
print(conflicts_json)
alliances_json = [{'source': a, 'target': b, 'value': w, 'type': 'ally'} for a, v in alliances.items() for b, w in v.items()]
print(alliances_json)

combined_links_json = conflicts_json + alliances_json

[{'id': '850'}, {'id': '432'}, {'id': '522'}, {'id': '680'}, {'id': '751'}, {'id': '439'}, {'id': '820'}, {'id': '160'}, {'id': '500'}, {'id': '615'}, {'id': '471'}, {'id': '93'}, {'id': '816'}, {'id': '2'}, {'id': '770'}, {'id': '95'}, {'id': '750'}, {'id': '200'}, {'id': '520'}, {'id': '800'}, {'id': '811'}, {'id': '663'}, {'id': '310'}, {'id': '626'}, {'id': '660'}, {'id': '900'}, {'id': '130'}, {'id': '713'}, {'id': '365'}, {'id': '640'}, {'id': '690'}, {'id': '812'}, {'id': '135'}, {'id': '91'}, {'id': '475'}, {'id': '731'}, {'id': '92'}, {'id': '55'}, {'id': '645'}, {'id': '620'}, {'id': '531'}, {'id': '483'}, {'id': '510'}, {'id': '220'}, {'id': '339'}, {'id': '616'}, {'id': '600'}, {'id': '817'}, {'id': '210'}, {'id': '530'}, {'id': '775'}, {'id': '651'}, {'id': '732'}, {'id': '666'}, {'id': '678'}, {'id': '710'}, {'id': '630'}, {'id': '625'}, {'id': '700'}, {'id': '652'}, {'id': '352'}]
[{'source': '220', 'target': '800', 'value': 1, 'type': 'enemy'}, {'source': '220', 'target

In [10]:
ccodes = pd.read_csv('data/raw_data/COW_country_codes.csv')
ccodes.head()

,StateAbb,CCode,StateNme
0,USA,2,United States of America
1,CAN,20,Canada
2,BHM,31,Bahamas
3,CUB,40,Cuba
4,CUB,40,Cuba


In [12]:
ccode_converter = {}

for i, row in ccodes.iterrows():
    if row['CCode'] not in ccode_converter:
        ccode_converter[row['CCode']]={'abb': row['StateAbb'], 'name': row['StateNme']}
    
print(ccode_converter)

{2: {'abb': 'USA', 'name': 'United States of America'}, 20: {'abb': 'CAN', 'name': 'Canada'}, 31: {'abb': 'BHM', 'name': 'Bahamas'}, 40: {'abb': 'CUB', 'name': 'Cuba'}, 41: {'abb': 'HAI', 'name': 'Haiti'}, 42: {'abb': 'DOM', 'name': 'Dominican Republic'}, 51: {'abb': 'JAM', 'name': 'Jamaica'}, 52: {'abb': 'TRI', 'name': 'Trinidad and Tobago'}, 53: {'abb': 'BAR', 'name': 'Barbados'}, 54: {'abb': 'DMA', 'name': 'Dominica'}, 55: {'abb': 'GRN', 'name': 'Grenada'}, 56: {'abb': 'SLU', 'name': 'St. Lucia'}, 57: {'abb': 'SVG', 'name': 'St. Vincent and the Grenadines'}, 58: {'abb': 'AAB', 'name': 'Antigua & Barbuda'}, 60: {'abb': 'SKN', 'name': 'St. Kitts and Nevis'}, 70: {'abb': 'MEX', 'name': 'Mexico'}, 80: {'abb': 'BLZ', 'name': 'Belize'}, 90: {'abb': 'GUA', 'name': 'Guatemala'}, 91: {'abb': 'HON', 'name': 'Honduras'}, 92: {'abb': 'SAL', 'name': 'El Salvador'}, 93: {'abb': 'NIC', 'name': 'Nicaragua'}, 94: {'abb': 'COS', 'name': 'Costa Rica'}, 95: {'abb': 'PAN', 'name': 'Panama'}, 100: {'abb'

In [13]:
folder = 'data/formatted_data/'
with open(folder + 'nodes.json', 'w') as f1,\
    open(folder + 'conflicts.json', 'w') as f2,\
    open(folder + 'alliances.json', 'w') as f3,\
    open(folder + 'ccode_converter.json', 'w') as f4, \
    open(folder + 'combined_links.json', 'w') as f5:
        f1.write(json.dumps(nodes_json))
        f2.write(json.dumps(conflicts_json))
        f3.write(json.dumps(alliances_json))
        f4.write(json.dumps(ccode_converter))
        f5.write(json.dumps(combined_links_json))